In [1]:
import pandas as pd

In [2]:
def_df = pd.read_csv("/Users/alyssaliu/Desktop/StatsM148/Event Definitions.csv")
df = pd.read_csv("/Users/alyssaliu/Desktop/StatsM148/export.csv")

In [3]:
def_df

,event_name,journey_id,event_definition_id,milestone_number,stage
0,application_phone_approved,1,15,1.0,Apply for Credit
1,application_phone_declined,1,16,NaN,Apply for Credit
2,application_phone_pending,1,17,NaN,Apply for Credit
3,application_web_approved,1,12,1.0,Apply for Credit
4,application_web_declined,1,13,NaN,Apply for Credit
5,application_web_pending,1,14,NaN,Apply for Credit
6,application_web_submit,1,3,NaN,Apply for Credit
7,application_web_view,1,19,NaN,Apply for Credit
8,account_activitation,1,29,3.0,Credit Account
9,account_fraud_review,1,37,NaN,Credit Account


In [4]:
print(df.shape)
df.head()

(64911906, 6)


,customer_id,account_id,ed_id,event_name,event_timestamp,journey_steps_until_end
0,-784961211,1773350293,12,application_web_approved,2023-03-22T08:45:22.000Z,1
1,-784961211,1773350293,19,application_web_view,2023-03-22T13:32:10.000Z,2
2,-784961211,1773350293,19,application_web_view,2023-03-22T13:32:10.000Z,3
3,-784961211,1773350293,19,application_web_view,2023-03-22T13:32:10.000Z,4
4,-784961211,1773350293,19,application_web_view,2023-03-22T13:32:10.000Z,5


In [5]:
df['event_timestamp'] = pd.to_datetime(df['event_timestamp']) # convert to pd datetime

## First deal with removing duplicate events

Ben said these could be ignored

In [6]:
# remove duplicate rows - don't factor in journey_steps_until_end because this counts up even for duplicate rows
df_dropped = df.drop(['journey_steps_until_end'], axis = 1)
df_dropped = df_dropped.drop_duplicates()
# df_dropped = df.drop(['journey_steps_until_end'], axis = 1)
df_dropped.head(10)

,customer_id,account_id,ed_id,event_name,event_timestamp
0,-784961211,1773350293,12,application_web_approved,2023-03-22 08:45:22+00:00
1,-784961211,1773350293,19,application_web_view,2023-03-22 13:32:10+00:00
14,-784961211,1773350293,3,application_web_submit,2023-03-22 13:32:10+00:00
15,-784961211,1773350293,2,campaign_click,2023-03-22 14:45:22+00:00
16,-784961211,1773350293,19,application_web_view,2023-07-27 14:57:56+00:00
21,-784961211,1773350293,19,application_web_view,2023-08-29 16:01:06+00:00
24,15849251,383997507,4,browse_products,2021-11-04 14:11:15+00:00
25,15849251,383997507,4,browse_products,2021-11-04 14:11:29+00:00
26,15849251,383997507,4,browse_products,2021-11-04 14:12:10+00:00
27,15849251,383997507,4,browse_products,2021-11-04 14:12:21+00:00


In [7]:
print(df_dropped.shape)
df.shape[0] - df_dropped.shape[0] # number of duplicate rows removed ~8 million

(56853035, 5)


8058871

In [8]:
df_dropped['journey_steps_until_end'] = df_dropped.groupby(['customer_id', 'account_id']).cumcount() + 1 # add journey_steps_until_end back in

In [9]:
df_dropped.reset_index(drop=True, inplace=True)
df_dropped.head(10)

,customer_id,account_id,ed_id,event_name,event_timestamp,journey_steps_until_end
0,-784961211,1773350293,12,application_web_approved,2023-03-22 08:45:22+00:00,1
1,-784961211,1773350293,19,application_web_view,2023-03-22 13:32:10+00:00,2
2,-784961211,1773350293,3,application_web_submit,2023-03-22 13:32:10+00:00,3
3,-784961211,1773350293,2,campaign_click,2023-03-22 14:45:22+00:00,4
4,-784961211,1773350293,19,application_web_view,2023-07-27 14:57:56+00:00,5
5,-784961211,1773350293,19,application_web_view,2023-08-29 16:01:06+00:00,6
6,15849251,383997507,4,browse_products,2021-11-04 14:11:15+00:00,1
7,15849251,383997507,4,browse_products,2021-11-04 14:11:29+00:00,2
8,15849251,383997507,4,browse_products,2021-11-04 14:12:10+00:00,3
9,15849251,383997507,4,browse_products,2021-11-04 14:12:21+00:00,4


In [10]:
df_dropped.dtypes

customer_id                              int64
account_id                               int64
ed_id                                    int64
event_name                              object
event_timestamp            datetime64[ns, UTC]
journey_steps_until_end                  int64
dtype: object

## Look into removing incomplete customers

Determining whether a customer is unsuccessful (quit) or incomplete (the data was just cut off, so we don't know if they ended up having a successful journey or not) 

according to Ben: it is a bit tricky to know if it’s a ‘quit’ or an ‘incomplete’.  What we typically look for is 60+ days since an approval, after that point we have VERY few people order if they haven’t yet.  Here’s our activation curve as of late as an example:

![Image Description](cum_activation.png)

I will create a dataframe for customers who have had 70+ days since activation - to weed out incomplete customers

In [11]:
# get count of account_activations for each customer/account pair
activation_counts = df_dropped.groupby(['customer_id', 'account_id'])['event_name'].apply(lambda x: x.str.count('account_activitation').sum())
activation_counts

customer_id  account_id 
-2147483541   2059949086    0
-2147481037  -1245770865    0
-2147474335  -1929965268    1
-2147474305   1098145658    0
-2147472167   889210381     1
                           ..
 2147480051  -148210516     0
 2147480182  -1286764612    0
 2147480920   496493954     0
 2147482120  -488077144     0
 2147483471   1556733004    0
Name: event_name, Length: 1735767, dtype: int64

In [12]:
# get count of account_activation occurrences
activation_counts.value_counts()

event_name
0    1319326
1     414268
2       2013
3        160
Name: count, dtype: int64

In [13]:
# just get those with multiple activations
multiple_activations = activation_counts[activation_counts > 1]
multiple_activations.value_counts()

event_name
2    2013
3     160
Name: count, dtype: int64

In [14]:
mult_act_cust = list(multiple_activations.index) # list of tuples of (customer_id, account_id) with multiple activations

# create the filter for dataframe rows that correspond to customers with multiple activations
filt = ~df_dropped.apply(lambda row: (row['customer_id'], row['account_id']) in mult_act_cust, axis=1)
df_filtered = df_dropped[filt] # remove customers with multiple activations

In [15]:
df_filtered

,customer_id,account_id,ed_id,event_name,event_timestamp,journey_steps_until_end
0,-784961211,1773350293,12,application_web_approved,2023-03-22 08:45:22+00:00,1
1,-784961211,1773350293,19,application_web_view,2023-03-22 13:32:10+00:00,2
2,-784961211,1773350293,3,application_web_submit,2023-03-22 13:32:10+00:00,3
3,-784961211,1773350293,2,campaign_click,2023-03-22 14:45:22+00:00,4
4,-784961211,1773350293,19,application_web_view,2023-07-27 14:57:56+00:00,5
...,...,...,...,...,...,...
56853030,-865005459,-983311387,29,account_activitation,2021-05-14 00:00:00+00:00,39
56853031,-865005459,-983311387,5,view_cart,2021-05-15 09:27:47+00:00,40
56853032,-865005459,-983311387,24,campaignemail_clicked,2021-05-15 14:27:33+00:00,41
56853033,-865005459,-983311387,27,account_downpaymentcleared,2021-05-16 00:00:00+00:00,42


#### Now check account_activation counts - should all be either 1 or 0

In [16]:
# now make sure all remaining customer/account combinations have only 0 or 1 account_activations
df_filtered.groupby(['customer_id', 'account_id'])['event_name'].apply(lambda x: x.str.count('account_activitation').sum()).value_counts()

#### It seems that multiple account activations just indicates multiple journeys under the same customer_id and account_id which is interesting

Because such a small portion of data was removed by removing customer with multiple activations, we will proceed with df_filtered

In [ ]:
df_dropped[['customer_id', 'account_id']].drop_duplicates().shape # 1735767 unique combinations of customer/account id

(1735767, 2)

In [ ]:
print(2173 / 1735767) # 0.001% of customer/account combinations would be removed if we removed those with multiple account activations
238751 / df_dropped.shape[0] # 0.4% of dataset would be dropped if we removed customers with multiple activations

0.0012518961358292905


0.004199441595334356

Because such a small portion of the dataset has multiple activations, we will drop these rows for simplicity and for the sake of keeping the data uniform.

We will proceed with attempting to remove "incomplete" customers in the data

In [ ]:
df_filtered

,customer_id,account_id,ed_id,event_name,event_timestamp,journey_steps_until_end
0,-784961211,1773350293,12,application_web_approved,2023-03-22 08:45:22+00:00,1
1,-784961211,1773350293,19,application_web_view,2023-03-22 13:32:10+00:00,2
2,-784961211,1773350293,3,application_web_submit,2023-03-22 13:32:10+00:00,3
3,-784961211,1773350293,2,campaign_click,2023-03-22 14:45:22+00:00,4
4,-784961211,1773350293,19,application_web_view,2023-07-27 14:57:56+00:00,5
...,...,...,...,...,...,...
99995,248264948,-136920922,11,add_to_cart,2021-04-22 12:28:15+00:00,12
99996,248264948,-136920922,4,browse_products,2021-04-22 12:30:31+00:00,13
99997,248264948,-136920922,5,view_cart,2021-04-22 12:30:48+00:00,14
99998,248264948,-136920922,1,promotion_created,2021-04-22 13:01:07.340000+00:00,15


## Now onto data cleaning - turning data into wide format

In [ ]:
# Group by 'customer_id' and aggregate the other columns into lists
df_grouped_cust_acct = df_filtered.groupby(['customer_id', 'account_id']).agg({
    'ed_id': list,
    'event_name': list,
    'event_timestamp': list,
    'journey_steps_until_end': list,
}).reset_index()

## add rows on account activation status

In [ ]:
# 1 if row activated account, 0 if not
df_grouped_cust_acct['account_activation'] = df_grouped_cust_acct['event_name'].apply(lambda x: 1 if 'account_activitation' in x else 0)

# 1 if row placed order, 0 if did not
df_grouped_cust_acct['place_order'] = df_grouped_cust_acct['event_name'].apply(lambda x: 1 if any([i in x for i in ['place_order_web', 'place_order_phone']]) else 0)

In [ ]:
activated_no_order = df_grouped_cust_acct[(df_grouped_cust_acct['account_activation'] == 1) & (df_grouped_cust_acct['place_order'] == 0)][['customer_id', 'account_id']]
activated_no_order.head()

,customer_id,account_id,ed_id,event_name,event_timestamp,journey_steps_until_end,account_activation,place_order
11,-2128292400,1711087044,"[4, 4, 4, 2, 19, 19, 3, 19, 19, 3, 19, 19, 12,...","[browse_products, browse_products, browse_prod...","[2021-01-12 22:09:33+00:00, 2021-01-12 22:10:5...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",1,0
21,-2116277535,-1178626321,"[12, 3, 19, 29, 24, 24, 24]","[application_web_approved, application_web_sub...","[2023-04-02 17:11:34+00:00, 2023-04-02 22:10:3...","[1, 2, 3, 4, 5, 6, 7]",1,0
93,-1984155795,-1325351971,"[2, 19, 19, 19, 19, 19, 19, 3, 19, 12, 4, 11, ...","[campaign_click, application_web_view, applica...","[2022-05-17 06:00:00+00:00, 2022-05-17 18:37:4...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",1,0
131,-1918117112,-139799356,"[19, 19, 19, 19, 19, 19, 3, 19, 19, 19, 19, 19...","[application_web_view, application_web_view, a...","[2021-09-26 14:39:21+00:00, 2021-09-26 14:39:3...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",1,0
187,-1817952213,-2122269252,"[29, 2, 19, 19, 19, 19, 19, 19, 19, 3, 19, 12,...","[account_activitation, campaign_click, applica...","[2021-12-05 00:00:00+00:00, 2021-12-05 06:00:0...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",1,0


### Now proceed with removing incomplete customers - those who activated within the last 60 days

In [ ]:
cutoff_date = df_dropped['event_timestamp'].max() # extract last date in dataset

activation_events = df_filtered[df_filtered['event_name'] == 'account_activitation'] # get account activations

# Merge to keep only activation events where the order wasn't shipped
df_merged = activation_events.merge(activated_no_order, on=['customer_id', 'account_id'])

# Group by 'customer_id' and 'account_id' to find the latest activation timestamp for each
activation_times = df_merged.groupby(['customer_id', 'account_id'])['event_timestamp'].max().reset_index()

# Calculate the number of days since activation for each customer/account pair
activation_times['days_since_activation'] = (cutoff_date - activation_times['event_timestamp']).dt.days

# Filter out rows where 'days_since_activation' is greater than 60
incomplete_cust_df = activation_times[activation_times['days_since_activation'] <= 60]


In [ ]:
incomplete_cust_df

,customer_id,account_id,event_timestamp,days_since_activation
5,-1590025467,1010220917,2023-08-29 00:00:00+00:00,21
27,-752043218,1897166153,2023-08-13 00:00:00+00:00,37
41,811457,1733237090,2023-09-15 00:00:00+00:00,4
42,43411333,771796683,2023-09-12 00:00:00+00:00,7
45,158128232,-1301093162,2023-08-01 00:00:00+00:00,49
67,1401055229,609339996,2023-08-03 00:00:00+00:00,47
75,1929486631,316446370,2023-07-27 00:00:00+00:00,54


In [ ]:
incomplete_cust = incomplete_cust_df[['customer_id', 'account_id']] # extract just customer_id and account_id

# drop rows where match incomplete_cust
cleaned_df = df_grouped_cust_acct[~df_grouped_cust_acct[['customer_id', 'account_id']].isin(incomplete_cust).all(axis=1)]

In [ ]:
cleaned_df

,customer_id,account_id,ed_id,event_name,event_timestamp,journey_steps_until_end,account_activation,place_order
0,-2147206560,2082689427,"[12, 1, 19, 5, 11, 3, 4, 6, 4]","[application_web_approved, promotion_created, ...","[2023-05-02 20:20:18+00:00, 2023-05-02 20:58:1...","[1, 2, 3, 4, 5, 6, 7, 8, 9]",0,0
1,-2145360520,1467252181,"[19, 19, 19, 19, 19, 19, 19, 3, 19, 12, 4, 4, ...","[application_web_view, application_web_view, a...","[2022-01-08 00:40:01+00:00, 2022-01-08 00:40:0...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",0,0
2,-2144991170,-2081722203,"[19, 19, 19, 19, 19, 19, 3, 19, 12, 4, 4, 4, 4...","[application_web_view, application_web_view, a...","[2022-09-19 14:47:59+00:00, 2022-09-19 14:48:0...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",0,0
3,-2142086624,-484534102,"[2, 12, 4, 4, 4, 5, 6, 1, 1, 6, 24, 1, 21, 1, ...","[campaign_click, application_web_approved, bro...","[2021-07-07 06:00:00+00:00, 2021-07-07 18:58:0...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",0,0
4,-2140007449,-660750512,"[19, 19, 19, 19, 19, 19, 19, 19, 3, 19, 12, 4,...","[application_web_view, application_web_view, a...","[2022-11-09 14:52:21+00:00, 2022-11-09 14:52:2...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",0,0
...,...,...,...,...,...,...,...,...
2627,2144404856,1842029764,"[19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 3...","[application_web_view, application_web_view, a...","[2022-06-29 19:56:08+00:00, 2022-06-29 19:56:1...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",1,1
2628,2144944055,386098939,"[19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 1...","[application_web_view, application_web_view, a...","[2022-11-24 21:02:25+00:00, 2022-11-24 21:02:2...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",1,1
2629,2145371253,382731392,"[19, 19, 19, 19, 3, 3, 12, 4, 4, 4, 4, 4, 4, 4...","[application_web_view, application_web_view, a...","[2022-01-24 12:00:46+00:00, 2022-01-24 12:01:1...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",0,0
2630,2146406955,-1661471236,"[4, 4, 4, 4, 4, 11, 5, 1, 1, 5, 24, 5, 5, 5, 1...","[browse_products, browse_products, browse_prod...","[2020-12-26 08:04:24+00:00, 2020-12-26 08:48:4...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",1,1


In [47]:
df_grouped.to_pickle("wide_format_data_no_duplicates.pkl")

## Old wide format data

In [28]:
# Group by 'customer_id' and aggregate the other columns into lists
df_grouped = df_dropped.groupby('customer_id').agg({
    'account_id': list,
    'ed_id': list,
    'event_name': list,
    'event_timestamp': list,
    'journey_steps_until_end': list,
}).reset_index()

In [29]:
df_grouped.head(2)

,customer_id,account_id,ed_id,event_name,event_timestamp,journey_steps_until_end
0,-2147483541,"[2059949086, 2059949086, 2059949086, 205994908...","[12, 5, 4, 11, 5, 6, 1, 4, 11, 4, 11, 5, 6, 5,...","[application_web_approved, view_cart, browse_p...","[2021-10-05 08:11:26+00:00, 2021-10-05 08:17:1...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,-2147481037,"[-1245770865, -1245770865, -1245770865, -12457...","[12, 1, 2, 5, 11, 4, 6]","[application_web_approved, promotion_created, ...","[2023-08-26 12:08:34+00:00, 2023-08-26 17:18:4...","[1, 2, 3, 4, 5, 6, 7]"


In [30]:
df_grouped.shape

(1665688, 6)

In [31]:
df_grouped['account_id'] = df_grouped['account_id'].apply(lambda x: list(set(x)))

In [32]:
df_grouped.head()

,customer_id,account_id,ed_id,event_name,event_timestamp,journey_steps_until_end
0,-2147483541,[2059949086],"[12, 5, 4, 11, 5, 6, 1, 4, 11, 4, 11, 5, 6, 5,...","[application_web_approved, view_cart, browse_p...","[2021-10-05 08:11:26+00:00, 2021-10-05 08:17:1...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,-2147481037,[-1245770865],"[12, 1, 2, 5, 11, 4, 6]","[application_web_approved, promotion_created, ...","[2023-08-26 12:08:34+00:00, 2023-08-26 17:18:4...","[1, 2, 3, 4, 5, 6, 7]"
2,-2147474335,[-1929965268],"[2, 12, 19, 3, 4, 4, 19, 4, 1, 11, 4, 5, 11, 4...","[campaign_click, application_web_approved, app...","[2023-04-14 01:42:50+00:00, 2023-04-14 19:42:5...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
3,-2147474305,[1098145658],"[6, 19, 19, 19, 19, 19, 19, 19, 1, 4, 19, 19, ...","[begin_checkout, application_web_view, applica...","[2021-11-28 22:27:11+00:00, 2021-11-28 22:28:0...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
4,-2147472167,[889210381],"[12, 19, 3, 4, 4, 1, 1, 11, 5, 4, 11, 5, 4, 6,...","[application_web_approved, application_web_vie...","[2023-02-19 08:23:54+00:00, 2023-02-19 14:21:5...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


In [33]:
df_grouped.to_pickle("wide_format_data_no_duplicates.pkl")

In [12]:
# df_grouped.to_csv('wide_format_df.csv')
# this doesn't preserve lists in the columns, turns them into strings